In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd

# --- आर्थिक वर्ष २०८१/८२ (2024/25) को लागि आयकर दरहरू ---
# स्रोत: नेपाल सरकार, अर्थ मन्त्रालय
TAX_SLABS = {
    'अविवाहित': [
        {'limit': 500000, 'rate': 0.01, 'base_tax': 0},
        {'limit': 700000, 'rate': 0.10, 'base_tax': 5000},
        {'limit': 1000000, 'rate': 0.20, 'base_tax': 25000},
        {'limit': 2000000, 'rate': 0.30, 'base_tax': 85000},
        {'limit': 5000000, 'rate': 0.36, 'base_tax': 385000},
        {'limit': float('inf'), 'rate': 0.39, 'base_tax': 1465000}
    ],
    'विवाहित': [
        {'limit': 600000, 'rate': 0.01, 'base_tax': 0},
        {'limit': 800000, 'rate': 0.10, 'base_tax': 6000},
        {'limit': 1100000, 'rate': 0.20, 'base_tax': 26000},
        {'limit': 2000000, 'rate': 0.30, 'base_tax': 86000},
        {'limit': 5000000, 'rate': 0.36, 'base_tax': 356000},
        {'limit': float('inf'), 'rate': 0.39, 'base_tax': 1436000}
    ]
}

def calculate_tax(annual_income, marital_status, has_ssf, pf_cit_contribution=0):
    """
    नेपालको नियम अनुसार वार्षिक आयकर गणना गर्दछ।
    """
    # SSF/PF/CIT योगदानलाई करयोग्य आयबाट घटाउने
    # अधिकतम छुट: आयको १/३ वा रु ५,००,००० (जुन कम हुन्छ)
    max_deduction = min(annual_income / 3, 500000)
    deductible_contribution = min(pf_cit_contribution, max_deduction)
    taxable_income = annual_income - deductible_contribution
    
    if taxable_income <= 0:
        return 0, taxable_income

    slabs = TAX_SLABS[marital_status]
    annual_tax = 0
    
    # यदि SSF छ भने १% सामाजिक सुरक्षा कर लाग्दैन
    start_index = 1 if has_ssf and taxable_income > slabs[0]['limit'] else 0

    if has_ssf and taxable_income <= slabs[0]['limit']:
        return 0, taxable_income

    previous_limit = 0
    for i in range(start_index, len(slabs)):
        slab = slabs[i]
        if taxable_income > previous_limit:
            if taxable_income <= slab['limit']:
                taxable_at_this_slab = taxable_income - previous_limit
                annual_tax = slab['base_tax'] + (taxable_at_this_slab * slab['rate'])
                break
        previous_limit = slab['limit']
        
    return annual_tax, taxable_income

# --- विजेटहरू (UI Components) ---
style = {'description_width': 'initial'}
monthly_income_input = widgets.FloatText(
    description="मासिक कुल आय (रु):", style=style, value=60000
)
marital_status_toggle = widgets.ToggleButtons(
    options=['अविवाहित', 'विवाहित'], description='वैवाहिक स्थिति:', style=style
)
contribution_choice = widgets.RadioButtons(
    options=['कुनै योगदान छैन', 'सामाजिक सुरक्षा कोष (SSF)', 'सञ्चय कोष/नागरिक लगानी कोष (PF/CIT)'],
    description='योगदान प्रकार:', style=style,
    layout={'width': 'max-content'}
)
contribution_amount_input = widgets.FloatText(
    description="मासिक PF/CIT योगदान (रु):", style=style, value=0, disabled=True
)
calculate_button = widgets.Button(
    description="कर गणना गर्नुहोस्", button_style='success', icon='calculator'
)
output_area = widgets.Output()

def on_contribution_choice_change(change):
    """PF/CIT छनोट गर्दा मात्र योगदान रकम बाकस देखाउने।"""
    if change.new == 'सञ्चय कोष/नागरिक लगानी कोष (PF/CIT)':
        contribution_amount_input.disabled = False
    else:
        contribution_amount_input.disabled = True
        contribution_amount_input.value = 0

contribution_choice.observe(on_contribution_choice_change, names='value')

def on_calculate_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        
        monthly_income = monthly_income_input.value
        marital_status = marital_status_toggle.value
        has_ssf = (contribution_choice.value == 'सामाजिक सुरक्षा कोष (SSF)')
        monthly_pf_cit = contribution_amount_input.value if contribution_choice.value == 'सञ्चय कोष/नागरिक लगानी कोष (PF/CIT)' else 0
        
        # वार्षिक हिसाब
        annual_income = monthly_income * 12
        annual_pf_cit = monthly_pf_cit * 12

        annual_tax, taxable_income = calculate_tax(annual_income, marital_status, has_ssf, annual_pf_cit)
        monthly_tax = annual_tax / 12
        
        # परिणामलाई DataFrame मा देखाउने
        report_data = {
            "विवरण": [
                "कुल वार्षिक आय",
                "कुल करयोग्य आय",
                "कुल वार्षिक कर",
                "औसत मासिक कर"
            ],
            "रकम (रु.)": [
                f"{annual_income:,.2f}",
                f"{taxable_income:,.2f}",
                f"{annual_tax:,.2f}",
                f"{monthly_tax:,.2f}"
            ]
        }
        df = pd.DataFrame(report_data)
        
        display(HTML("<h3>तपाईंको कर विवरण (आ.व. २०८१/८२)</h3>"))
        display(df)
        
calculate_button.on_click(on_calculate_button_clicked)

# --- UI प्रदर्शन ---
form = widgets.VBox([
    monthly_income_input,
    marital_status_toggle,
    contribution_choice,
    contribution_amount_input,
    calculate_button
])

display(HTML("<h2>नेपाल आयकर क्याल्कुलेटर (FY 2081/82)</h2>"))
display(HTML("<p>आफ्नो मासिक आय र अन्य विवरणहरू प्रविष्ट गरी करको हिसाब गर्नुहोस्।</p>"))
display(form)
display(output_area)

Output()

In [3]:
!pip install ipycanvas

In [4]:
import math
from ipycanvas import Canvas

# --- क्यानभास सिर्जना ---
# ६००x६०० पिक्सेलको क्यानभास बनाउने
canvas = Canvas(width=600, height=600)

def create_flag(ctx):
    """नेपालको झण्डाको बाहिरी भाग (pennants) बनाउँछ।"""
    
    # झण्डाको रङ र बोर्डरको मोटाई सेट गर्ने
    ctx.stroke_style = 'blue'
    ctx.fill_style = 'crimson' # नेपालको झण्डाको आधिकारिक रङ 'crimson red' हो
    ctx.line_width = 8 # बोर्डरको मोटाई
    
    ctx.begin_path()
    
    # झण्डाको आकार कोर्ने
    # Coordinates लाई JS कोडबाट मिलाइएको छ
    ctx.move_to(100, 50)     # १. माथिल्लो बिन्दु
    ctx.line_to(450, 250)    # २. माथिल्लो त्रिकोणको टुप्पो
    ctx.line_to(220, 250)    # ३. बीचको भित्री बिन्दु
    ctx.line_to(450, 450)    # ४. तल्लो त्रिकोणको टुप्पो
    ctx.line_to(100, 450)    # ५. तल्लो बिन्दु
    ctx.close_path()         # सुरुको बिन्दुमा जोड्ने
    
    ctx.fill()
    ctx.stroke()

def draw_moon(ctx):
    """माथिल्लो भागमा चन्द्रमा बनाउँछ।"""
    
    ctx.begin_path()
    ctx.line_width = 2
    ctx.stroke_style = 'white'
    ctx.fill_style = 'white'
    
    # JS कोडमा भएको चन्द्रमाको आकारलाई यहाँ उतारिएको छ
    ctx.move_to(210, 190)
    ctx.line_to(200, 170)
    ctx.line_to(185, 190)
    ctx.line_to(175, 170)
    ctx.line_to(160, 190)
    ctx.line_to(145, 170)
    ctx.line_to(135, 190)
    ctx.arc(172, 190, 37, 0, math.pi) # अर्ध-वृत्त
    
    ctx.fill()
    ctx.stroke()

def draw_sun(ctx):
    """तल्लो भागमा सूर्य बनाउँछ।"""
    
    ctx.begin_path()
    ctx.fill_style = 'white'
    ctx.stroke_style = 'white'
    ctx.line_width = 2

    # JS कोडमा भएको सूर्यको १२-कुने आकार
    ctx.move_to(180, 340) # Start point
    
    ctx.line_to(175, 350)
    ctx.line_to(155, 345)
    ctx.line_to(165, 360)
    ctx.line_to(145, 365)
    ctx.line_to(160, 375)
    ctx.line_to(145, 385)
    ctx.line_to(160, 395)
    ctx.line_to(150, 410)
    ctx.line_to(170, 408)
    ctx.line_to(180, 425) # Bottom point
    ctx.line_to(190, 410)
    ctx.line_to(208, 422)
    ctx.line_to(210, 405)
    ctx.line_to(230, 405)
    ctx.line_to(223, 390)
    ctx.line_to(242, 385)
    ctx.line_to(222, 372)
    ctx.line_to(235, 355)
    ctx.line_to(215, 355)
    ctx.line_to(210, 330)
    ctx.line_to(200, 350)

    ctx.close_path()
    ctx.fill()
    ctx.stroke()

# --- मुख्य कार्य ---
# क्यानभासमा सबै आकृतिहरू कोर्ने
print("====== नेपालको झण्डा ======")
create_flag(canvas)
draw_moon(canvas)
draw_sun(canvas)

# अन्तिममा क्यानभास प्रदर्शन गर्ने
display(canvas)

====== नेपालको झण्डा ======


Canvas(height=600, width=600)